In [8]:
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")

The file, autism.csv, contains the autism dataset, where
* vsae, Vineland Socialization Age Equivalent, is the dependent variable
* childid is the unique child identifier
* age, age at which vsae is measured, is a time variable
* sicdegp is the sequenced inventory of communication development expressive group, with 1 being low, 2 being median, and 3 being high

In [9]:
# read excel wb
df = pd.read_csv("autism.csv")

df["Age"] = df["age"] - 2
df['Age2'] = df["Age"] **2

df.loc[df['sicdegp'] == 3, 'Grp'] = 0
df.loc[df['sicdegp'] == 1, 'Grp'] = 1
df.loc[df['sicdegp'] == 2, 'Grp'] = 2

df['Grp'] = df['Grp'].astype('category')

# drop rows with missing values
df = df.dropna()

df.head()
df.tail()

,age,vsae,sicdegp,childid,Age,Age2,Grp
0,2,6.0,3,1,0,0,0.0
1,3,7.0,3,1,1,1,0.0
2,5,18.0,3,1,3,9,0.0
3,9,25.0,3,1,7,49,0.0
4,13,27.0,3,1,11,121,0.0


,age,vsae,sicdegp,childid,Age,Age2,Grp
607,5,6.0,1,202,3,9,1.0
608,13,12.0,1,202,11,121,1.0
609,2,4.0,1,210,0,0,1.0
610,3,25.0,1,210,1,1,1.0
611,9,130.0,1,210,7,49,1.0


[a] Use statsmodels.formula.api.mixedlm() to fit the following models (using REML). Let your
code report the log-likelihood achieved for fitting each model.
* model A: vsae ∼Age + Age2+ Grp + Age ∗Grp + Age2∗Grp + (0 + Age + Age2|childid)
    * vc formula = { }
* model B: vsae ∼Age + Age2+ Grp + Age ∗Grp + Age2∗Grp + (0 + Age|childid)
    * vc formula = {‘Age’:‘0 + Age’, ‘Grp’:‘0 + Grp’}
* model C: vsae ∼Age + Age2+ Grp + Age ∗Grp + (0 + Age + Age2|childid)
    * vc formula = { }

Note that
* Age = age −2
* Grp = sicdegp is a categorical variable, with sicdegp = 3 as the refence level in these models


In [10]:
# Define formulas
formula_a = 'vsae ~ Age + Age2 + Grp + Age*Grp + Age2*Grp'

# Fit models and report log-likelihoods
model_a = smf.mixedlm(formula_a, df, 
                      groups = df['childid'], 
                      re_formula = "~ 0 + Age + Age2", 
                      vc_formula = {})
result_a = model_a.fit(reml=False)
print(f"Log-likelihood for model A: {result_a.llf:.4f}")

Log-likelihood for model A: -2305.2220


In [11]:
formula_b = 'vsae ~ Age + Age2 + Grp + Age*Grp + Age2*Grp'

model_b = smf.mixedlm(formula_b, df, 
                      groups = df['childid'], 
                      re_formula = "~ 0 + Age", 
                      vc_formula = {'Age':'0 + Age', 'Grp':'0 + Grp'})
result_b = model_b.fit(reml=False)
print(f"Log-likelihood for model B: {result_b.llf:.4f}")


Log-likelihood for model B: -2346.8312


In [12]:
formula_c = 'vsae ~ Age + Age2 + Grp + Age*Grp'

model_c = smf.mixedlm(formula_c, df, 
                      groups = df['childid'], 
                      re_formula = "~ 0 + Age + Age2", 
                      vc_formula = {})
result_c = model_c.fit(reml=False)
print(f"Log-likelihood for model C: {result_c.llf:.4f}")

Log-likelihood for model C: -2306.1569


[b] Let your code perform model selection and identify the best model (which is Model C). Let your
code report the test statistics and the corresponding p-values.

In [13]:
results = pd.DataFrame({
    'model': ['A', 'B', 'C'],
    'AIC': [result_a.aic, result_b.aic, result_c.aic],
    'BIC': [result_a.bic, result_b.bic, result_c.bic]
})

best_model = results.loc[results['AIC'].idxmin(), 'model']
print(f"The best model is model {best_model}.")

The best model is model C.


In [15]:
type(result_c)

statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper

In [14]:
# Report test statistics and p-values for model C
result_c.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
          Mixed Linear Model Regression Results
=========================================================
Model:             MixedLM Dependent Variable: vsae      
No. Observations:  610     Method:             ML        
No. Groups:        158     Scale:              38.4864   
Min. group size:   1       Log-Likelihood:     -2306.1569
Max. group size:   5       Converged:          Yes       
Mean group size:   3.9                                   
---------------------------------------------------------
               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------
Intercept      13.463    0.779 17.273 0.000 11.935 14.990
Grp[T.1.0]     -4.987    1.035 -4.818 0.000 -7.015 -2.958
Grp[T.2.0]     -3.622    0.974 -3.717 0.000 -5.532 -1.712
Age             6.149    0.686  8.963 0.000  4.805  7.494
Age:Grp[T.1.0] -4.069    0.876 -4.644 0.000 -5.787 -2.352
Age:Grp[T.2.0] -3.496    0.825 -4.236 0.000 -5.114 -1.879
Age2            0.109    0.043  2.562 0.010  0.026  0.193
Age Var        14.199    0.476                           
Age x Age2 Cov -0.409    0.034                           
Age2 Var        0.125    0.005                           
=========================================================

"""